# Regularization \[8 points\]

<b><span style="color:#00CED1">Association:</span></b>
<span style="color:#00CED1">Otto-Friedrich University of Bamberg</span>
<span style="color:#00CED1">Chair of Explainable Machine Learning (xAI)</span>
<span style="color:#00CED1">Deep Learning Assignments</span>

<b><span style="color:#00CED1">Description:</span></b>
<span style="color:#00CED1">This notebook introduces regularization to our models. In particular we will learn about L2 regularization and dropout.</span>
<span style="color:#00CED1">Students will implement all of those and learn how each of them lead to different results.</span>

<span style="color:#00CED1"><b>Author:</b> Sebastian Doerrich</span>
<span style="color:#00CED1"><b>Copyright:</b> Copyright (c) 2022, Chair of Explainable Machine Learning (xAI), Otto-Friedrich University of Bamberg</span>
<span style="color:#00CED1"><b>Credits:</b> Christian Ledig, Sebastian Doerrich</span>
<span style="color:#00CED1"><b>License:</b> CC BY-SA</span>
<span style="color:#00CED1"><b>Version:</b> 1.0</span>
<span style="color:#00CED1"><b>Python:</b> Python 3</span>
<span style="color:#00CED1"><b>Maintainer:</b> Sebastian Doerrich</span>
<span style="color:#00CED1"><b>Email:</b> sebastian.doerrich@uni-bamberg.de</span>
<span style="color:#00CED1"><b>Status:</b> Production</span>

## Context
Welcome to this part of your fourth programming assignment.
In this part you will learn how to use regularization in your deep learning models.

## Motivation
Deep Learning models have so much flexibility and capacity that **overfitting can be a serious problem**, if the training dataset is not big enough. Sure your model might do well on the training set, but it **doesn't generalize to new examples** that it has never seen! Regularization helps to deal with this.

Let's get started!

## Instructions
- You will be using Python 3.
- After coding your function, run the cell right below it to check if your result is correct.

## Important Notes for Your Submission
Before submitting your assignment, please make sure you are not doing the following:

1. You have not added any _extra_ `print` statement(s) in the assignment.
2. You have not added any _extra_ code cell(s) in the assignment.
3. You have not changed any of the function parameters.
4. You are not using any global variables inside your graded exercises. Unless specifically instructed to do so, please refrain from it and use the local variables instead.
5. You are not changing the assignment code where it is not required, like creating _extra_ variables.

If you do any of the mentioned, our test scripts will fail and as a result you will receive **0 points** for the respective task.

## Table of Contents
- [0 - Import the Necessary Libraries](#0)
- [1 - Visualization](#1)
- [2 - Utils](#2)
- [3 - Problem Statement](#3)
- [4 - Data Loading](#4)
- [5 - Model](#5)
- [6 - Non-Regularized Model](#6)
    - [6.1 - Analyzing the Non-Regularized Model](#6-1)
- [7 - L2 Regularization](#7)
    - [7.1 - Cost Function](#7-1)
    - [7.2 - Backward Propagation with Regularization](#7-2)
    - [7.3 - Let’s now run the model with L2 regularization](#7-3)
    - [7.4 - Analyzing L2 regularization](#7-4)
- [8 - Dropout](#8)
    - [8.1 - Forward Propagation with Dropout](#8-1)
    - [8.2 - Backward Propagation with Dropout](#8-2)
    - [8.3 - Let’s now run the model with Dropout](#8-3)
    - [8.4 - Analyzing Dropout](#8-4)
- [9 - Conclusions](#9)
- [10 - End of Exercise](#10)

<a name='0'></a>
## 0 - Import the Necessary Libraries ##

In [ ]:
# Import packages
import numpy as np
import scipy.io
import warnings

import matplotlib.pyplot as plt
%matplotlib inline

#suppress warnings
warnings.filterwarnings('ignore')

<a id='1'></a>
## 1 - Visualization ##

These functions will be used for visualization. We provide those for you, so you can plot your results. You don't have to code anything here.

In [ ]:
class Visualization:
    """
    This class is used to collect all visualizations.
    """

    @staticmethod
    def visualize_the_data(train_X: np.ndarray, train_Y: np.ndarray, test_X: np.ndarray, test_Y: np.ndarray, scale=40, cmap=plt.cm.Spectral):
        """
        Visualize the samples.

        :param train_X: Training samples.
        :param train_Y: Training Labels.
        :param test_X: Test samples.
        :param test_Y: Test labels.
        :param scale: The marker size.
        :param cmap: The Colormap instance or registered colormap name used to map scalar data to colors.
        """

        # Create both plots
        _, (train_ax, test_ax) = plt.subplots(ncols=2, sharex=True, sharey=True, figsize=(32, 8))

        # Create plot for the training samples
        train_scatter = train_ax.scatter(train_X[0, :], train_X[1, :], c=train_Y, s=scale, cmap=cmap, label=[0, 1])
        train_ax.set_ylabel("Width of the pitch")
        train_ax.set_xlabel("Length of the pitch")
        train_ax.set_title("Training data")
        elements, labels = train_scatter.legend_elements()
        train_ax_legend = train_ax.legend(elements, ['Other Player', 'French Player'], title="Player Received the Ball")
        train_ax.add_artist(train_ax_legend)

        # Create the plot for the test samples
        test_scatter = test_ax.scatter(test_X[0, :], test_X[1, :], c=test_Y, s=scale, cmap=cmap, label=[0, 1])
        train_ax.set_ylabel("Width of the pitch")
        test_ax.set_xlabel("Length of the pitch")
        test_ax.set_title("Test data")
        elements, labels = test_scatter.legend_elements()
        test_ax_legend = test_ax.legend(elements, ['Other Player', 'French Player'], title="Player Received the Ball")
        test_ax.add_artist(test_ax_legend)

    @staticmethod
    def plot_the_cost(iterations: list, costs: list):
        """
        Plot the cost of the network during training.

        :param iterations: Training iterations in the form [0, 1000, 2000, ..., 15000].
        :param costs: Costs of the network for each training iteration.
        """

        # Plot the cost
        plt.plot(iterations, costs, label='cost')
        plt.ylabel('Cost')
        plt.xlabel('Iterations')
        plt.title("Training Curve")
        plt.legend()

    @staticmethod
    def plot_decision_boundaries(Z_train: np.ndarray, X_train: np.ndarray, Y_train: np.ndarray, Z_test: np.ndarray, X_test: np.ndarray, Y_test: np.ndarray):
        """
        Plot the decision boundary of the model.

        :param Z_train: Training Decision Function.
        :param X_train: Training amples.
        :param Y_train: Training Labels.
        :param Z_test: Test Decision Function.
        :param X_test: Test amples.
        :param Y_test: Test Labels.
        """

        # Create both plots
        _, (train_ax, test_ax) = plt.subplots(ncols=2, sharex=True, sharey=True, figsize=(32, 8))

        # Initialize the hyperparameters
        h = 0.01

        # Set min and max values and give it some padding
        x_min_train, x_max_train = X_train[0, :].min() - 1, X_train[0, :].max() + 1
        y_min_train, y_max_train = X_train[1, :].min() - 1, X_train[1, :].max() + 1

        x_min_test, x_max_test = X_test[0, :].min() - 1, X_test[0, :].max() + 1
        y_min_test, y_max_test = X_test[1, :].min() - 1, X_test[1, :].max() + 1

        # Generate a grid of points with distance h between them
        xx_train, yy_train = np.meshgrid(np.arange(x_min_train, x_max_train, h), np.arange(y_min_train, y_max_train, h))
        xx_test, yy_test = np.meshgrid(np.arange(x_min_test, x_max_test, h), np.arange(y_min_test, y_max_test, h))

        # Plot the decision boundary and training samples
        train_ax.set_ylabel("Width of the pitch")
        train_ax.set_xlabel("Length of the pitch")
        train_ax.set_title("Decision Boundary of the Model for the Training Data")

        train_ax.set_xlim([-0.75,0.40])
        train_ax.set_ylim([-0.75,0.65])

        train_ax.contourf(xx_train, yy_train, Z_train, cmap=plt.cm.Spectral)
        train_scatter = train_ax.scatter(X_train[0, :], X_train[1, :], c=Y_train, cmap=plt.cm.Spectral)

        elements, labels = train_scatter.legend_elements()
        train_ax_legend = train_ax.legend(elements, ['Other Player', 'French Player'], title="Player Received the Ball")
        train_ax.add_artist(train_ax_legend)

        # Plot the decision boundary and test samples
        test_ax.set_ylabel("Width of the pitch")
        test_ax.set_xlabel("Length of the pitch")
        test_ax.set_title("Decision Boundary of the Model for the Test Data")

        test_ax.set_xlim([-0.75,0.40])
        test_ax.set_ylim([-0.75,0.65])

        test_ax.contourf(xx_test, yy_test, Z_test, cmap=plt.cm.Spectral)
        test_scatter = test_ax.scatter(X_test[0, :], X_test[1, :], c=Y_test, cmap=plt.cm.Spectral)

        elements, labels = test_scatter.legend_elements()
        test_ax_legend = test_ax.legend(elements, ['Other Player', 'French Player'], title="Player Received the Ball")
        test_ax.add_artist(test_ax_legend)

<a id='2'></a>
## 2 - Utils ##

These functions will be used as helper functions throughout the entire notebook. We provide those for you, so you can focus on the regularization task at hand. You don't have to code anything here.

In [ ]:
class Utils:
    """
    This class collects helper functions for the notebook.
    """

    @staticmethod
    def sigmoid(x):
        """
        Compute the sigmoid of x

        Arguments:
        x -- A scalar or numpy array of any size.

        Return:
        s -- sigmoid(x)
        """
        s = 1/(1+np.exp(-x))
        return s

    @staticmethod
    def relu(x):
        """
        Compute the relu of x

        Arguments:
        x -- A scalar or numpy array of any size.

        Return:
        s -- relu(x)
        """
        s = np.maximum(0,x)

        return s

    @staticmethod
    def load_planar_dataset(seed):

        np.random.seed(seed)

        m = 400 # number of examples
        N = int(m/2) # number of points per class
        D = 2 # dimensionality
        X = np.zeros((m,D)) # data matrix where each row is a single example
        Y = np.zeros((m,1), dtype='uint8') # labels vector (0 for red, 1 for blue)
        a = 4 # maximum ray of the flower

        for j in range(2):
            ix = range(N*j,N*(j+1))
            t = np.linspace(j*3.12,(j+1)*3.12,N) + np.random.randn(N)*0.2 # theta
            r = a*np.sin(4*t) + np.random.randn(N)*0.2 # radius
            X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
            Y[ix] = j

        X = X.T
        Y = Y.T

        return X, Y

    @staticmethod
    def initialize_parameters(layer_dims):
        """
        Arguments:
        layer_dims -- python array (list) containing the dimensions of each layer in our network

        Returns:
        parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                        W1 -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                        b1 -- bias vector of shape (layer_dims[l], 1)
                        Wl -- weight matrix of shape (layer_dims[l-1], layer_dims[l])
                        bl -- bias vector of shape (1, layer_dims[l])

        Tips:
        - For example: the layer_dims for the "Planar Data classification model" would have been [2,2,1].
        This means W1's shape was (2,2), b1 was (1,2), W2 was (2,1) and b2 was (1,1). Now you have to generalize it!
        - In the for loop, use parameters['W' + str(l)] to access Wl, where l is the iterative integer.
        """

        np.random.seed(3)
        parameters = {}
        L = len(layer_dims) # number of layers in the network

        for l in range(1, L):
            parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) / np.sqrt(layer_dims[l-1])
            parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))

            assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
            assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))


        return parameters

    @staticmethod
    def forward_propagation(X, parameters):
        """
        Implements the forward propagation (and computes the loss) presented in Figure 2.

        Arguments:
        X -- input dataset, of shape (input size, number of examples)
        parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3":
                        W1 -- weight matrix of shape ()
                        b1 -- bias vector of shape ()
                        W2 -- weight matrix of shape ()
                        b2 -- bias vector of shape ()
                        W3 -- weight matrix of shape ()
                        b3 -- bias vector of shape ()

        Returns:
        loss -- the loss function (vanilla logistic loss)
        """

        # retrieve parameters
        W1 = parameters["W1"]
        b1 = parameters["b1"]
        W2 = parameters["W2"]
        b2 = parameters["b2"]
        W3 = parameters["W3"]
        b3 = parameters["b3"]

        # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
        Z1 = np.dot(W1, X) + b1
        A1 = Utils.relu(Z1)
        Z2 = np.dot(W2, A1) + b2
        A2 = Utils.relu(Z2)
        Z3 = np.dot(W3, A2) + b3
        A3 = Utils.sigmoid(Z3)

        cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)

        return A3, cache

    @staticmethod
    def backward_propagation(X, Y, cache):
        """
        Implement the backward propagation presented in figure 2.

        Arguments:
        X -- input dataset, of shape (input size, number of examples)
        Y -- true "label" vector (containing 0 if cat, 1 if non-cat)
        cache -- cache output from forward_propagation()

        Returns:
        gradients -- A dictionary with the gradients with respect to each parameter, activation and pre-activation variables
        """
        m = X.shape[1]
        (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache

        dZ3 = A3 - Y
        dW3 = 1./m * np.dot(dZ3, A2.T)
        db3 = 1./m * np.sum(dZ3, axis=1, keepdims = True)

        dA2 = np.dot(W3.T, dZ3)
        dZ2 = np.multiply(dA2, np.int64(A2 > 0))
        dW2 = 1./m * np.dot(dZ2, A1.T)
        db2 = 1./m * np.sum(dZ2, axis=1, keepdims = True)

        dA1 = np.dot(W2.T, dZ2)
        dZ1 = np.multiply(dA1, np.int64(A1 > 0))
        dW1 = 1./m * np.dot(dZ1, X.T)
        db1 = 1./m * np.sum(dZ1, axis=1, keepdims = True)

        gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                     "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                     "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}

        return gradients

    @staticmethod
    def update_parameters(parameters, grads, learning_rate):
        """
        Update parameters using gradient descent

        Arguments:
        parameters -- python dictionary containing your parameters:
                        parameters['W' + str(i)] = Wi
                        parameters['b' + str(i)] = bi
        grads -- python dictionary containing your gradients for each parameters:
                        grads['dW' + str(i)] = dWi
                        grads['db' + str(i)] = dbi
        learning_rate -- the learning rate, scalar.

        Returns:
        parameters -- python dictionary containing your updated parameters
        """

        n = len(parameters) // 2 # number of layers in the neural networks

        # Update rule for each parameter
        for k in range(n):
            parameters["W" + str(k+1)] = parameters["W" + str(k+1)] - learning_rate * grads["dW" + str(k+1)]
            parameters["b" + str(k+1)] = parameters["b" + str(k+1)] - learning_rate * grads["db" + str(k+1)]

        return parameters

    @staticmethod
    def predict(X, y, parameters, train=True):
        """
        This function is used to predict the results of a  n-layer neural network.

        Arguments:
        X -- data set of examples you would like to label
        parameters -- parameters of the trained model
        train -- train set given (boolean)

        Returns:
        p -- predictions for the given dataset X
        """

        m = X.shape[1]
        p = np.zeros((1,m), dtype=int)

        # Forward propagation
        a3, caches = Utils.forward_propagation(X, parameters)

        # convert probas to 0/1 predictions
        for i in range(0, a3.shape[1]):
            if a3[0,i] > 0.5:
                p[0,i] = 1
            else:
                p[0,i] = 0

        # print results
        if train:
            print("Accuracy of the model on the train set: " + str(np.mean((p[0,:] == y[0,:]))))

        else:
            print("Accuracy of the model on the test set: " + str(np.mean((p[0,:] == y[0,:]))))

        return p

    @staticmethod
    def compute_cost(a3, Y):
        """
        Implement the cost function

        Arguments:
        a3 -- post-activation, output of forward propagation
        Y -- "true" labels vector, same shape as a3

        Returns:
        cost - value of the cost function
        """
        m = Y.shape[1]
        logprobs = np.multiply(-np.log(a3, where=a3>0),Y) + np.multiply(-np.log(1 - a3, where=(1 - a3)>0), 1 - Y)
        cost = 1./m * np.nansum(logprobs)

        return cost

    @staticmethod
    def predict_dec(parameters, X):
        """
        Used for plotting decision boundary.

        Arguments:
        parameters -- python dictionary containing your parameters
        X -- input data of size (m, K)

        Returns
        predictions -- vector of predictions of our model (red: 0 / blue: 1)
        """

        # Predict using forward propagation and a classification threshold of 0.5
        a3, cache = Utils.forward_propagation(X, parameters)
        predictions = (a3>0.5)
        return predictions

    @staticmethod
    def load_planar_dataset(randomness, seed):

        np.random.seed(seed)

        m = 50
        N = int(m/2) # number of points per class
        D = 2 # dimensionality
        X = np.zeros((m,D)) # data matrix where each row is a single example
        Y = np.zeros((m,1), dtype='uint8') # labels vector (0 for red, 1 for blue)
        a = 2 # maximum ray of the flower

        for j in range(2):

            ix = range(N*j,N*(j+1))
            if j == 0:
                t = np.linspace(j, 4*3.1415*(j+1),N) #+ np.random.randn(N)*randomness # theta
                r = 0.3*np.square(t) + np.random.randn(N)*randomness # radius
            if j == 1:
                t = np.linspace(j, 2*3.1415*(j+1),N) #+ np.random.randn(N)*randomness # theta
                r = 0.2*np.square(t) + np.random.randn(N)*randomness # radius

            X[ix] = np.c_[r*np.cos(t), r*np.sin(t)]
            Y[ix] = j

        X = X.T
        Y = Y.T

        return X, Y

    @staticmethod
    def get_decision_boundary(model, X, y):
        # Set min and max values and give it some padding
        x_min, x_max = X[0, :].min() - 1, X[0, :].max() + 1
        y_min, y_max = X[1, :].min() - 1, X[1, :].max() + 1
        h = 0.01
        # Generate a grid of points with distance h between them
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
        # Predict the function value for the whole grid
        Z = model(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)

        return Z

<a id='3'></a>
## 3 - Problem Statement ##

You have just been hired as an AI expert by the French Football Corporation. They would like you to recommend positions where French's goal keeper should kick the ball so that the French team's players can then hit it with their head. This is illustrated in the figure below:

<div>
<img src="../img/field_kiank.png" width="700"/>
</div>

Within the figure you can see a Football field. The goal keeper kicks the ball in the air and the players of each team are fighting to hit the ball with their head.

The French Football Corporation gives you the following 2D dataset from French's past 10 games.

<a id='4'></a>
## 4 - Data Loading ##

First, we need to load our data. This step was already implemented for you below. You don’t have to code anything here.

In [ ]:
class DataLoader:
    """
    This class creates a data loader instance.
    """

    def __init__(self, file_path_data: str):
        """
        Constructor.

        :param file_path_data: Path to the folder containing the data entries.
        """

        self.file_path_data = file_path_data

        self.train_X, self.train_Y, self.test_X, self.test_Y = None, None, None, None

    def load_dataset(self):
        """
        Load the train and test set.

        :return:
            - train_X: Training samples.
            - train_Y: Training labels.
            - test_X: Test samples.
            - test_Y: Test labels.
        """

        if not self.train_X:
            data = scipy.io.loadmat(self.file_path_data)

            self.train_X = data['X'].T
            self.train_Y = data['y'].T
            self.test_X = data['Xval'].T
            self.test_Y = data['yval'].T

        return self.train_X, self.train_Y, self.test_X, self.test_Y

In [ ]:
# Run this cell to load and plot your data

# Load the dataset
data_path = '../data/data.mat'
dataloader = DataLoader(data_path)
train_X, train_Y, test_X, test_Y = dataloader.load_dataset()

# Plot the dataset
Visualization.visualize_the_data(train_X, train_Y, test_X, test_Y)

Each dot corresponds to a position on the football field where a football player has hit the ball with his/her head after the French goal keeper has shot the ball from the left side of the football field.
- If the dot is blue (purple for different color scheme), it means the French player managed to hit the ball with his/her head
- If the dot is red (pink for different color scheme), it means the other team's player hit the ball with their head


**Your goal is now**:
To use a deep learning model to find the positions on the field where the goalkeeper should kick the ball.


**First glance at the dataset**:
This dataset is a little noisy, but it looks like a diagonal line separating the upper left half (blue/purple) from the lower right half (red/pink) would work well.


**What you will do**:
You will first try a non-regularized model. Then you'll learn how to regularize it and decide which model you will choose to solve the French Football Corporation's problem.

<a name='5'></a>
## 5 - Model

You will use the following neural network (already implemented for you below). You don’t have to code anything here.
This model can be used:
- in *regularization mode* -- by setting the `lambd` input to a non-zero value. We use "`lambd`" instead of "`lambda`" because "`lambda`" is a reserved keyword in Python.
- in *dropout mode* -- by setting the `keep_prob` to a value less than one

You will first try the model without any regularization. Then, you will implement:
- *L2 regularization* -- functions: "`compute_cost_with_regularization()`" and "`backward_propagation_with_regularization()`"
- *Dropout* -- functions: "`forward_propagation_with_dropout()`" and "`backward_propagation_with_dropout()`"

In each part, we will run this model with the correct inputs so that it calls the functions you've implemented. Take a look at the code below to familiarize yourself with the model.

In [ ]:
def model(X: np.ndarray, Y: np.ndarray, learning_rate = 0.3, num_iterations = 30000, print_cost = True, lambd = 0, keep_prob = 1):
    """
    Implements a three-layer neural network: LINEAR->RELU->LINEAR->RELU->LINEAR->SIGMOID.

    :param X: input data, of shape (input size, number of examples)
    :param Y: true "label" vector (1 for blue dot / 0 for red dot), of shape (output size, number of examples)
    :param learning_rate: learning rate of the optimization
    :param num_iterations: number of iterations of the optimization loop
    :param print_cost: If True, print the cost every 10000 iterations
    :param lambd: regularization hyperparameter, scalar
    :param keep_prob: probability of keeping a neuron active during drop-out, scalar.

    :return:
        - parameters: parameters learned by the model. They can then be used to predict.
    """

    grads = {}
    costs = []                            # to keep track of the cost
    m = X.shape[1]                        # number of examples
    layers_dims = [X.shape[0], 20, 3, 1]

    # Initialize parameters dictionary.
    parameters = Utils.initialize_parameters(layers_dims)

    # Loop (gradient descent)

    for i in range(0, num_iterations):

        # Forward propagation: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID.
        if keep_prob == 1:
            a3, cache = Utils.forward_propagation(X, parameters)
        elif keep_prob < 1:
            a3, cache = forward_propagation_with_dropout(X, parameters, keep_prob)

        # Cost function
        if lambd == 0:
            cost = Utils.compute_cost(a3, Y)
        else:
            cost = compute_cost_with_regularization(a3, Y, parameters, lambd)

        # Backward propagation.
        assert (lambd == 0 or keep_prob == 1)   # it is possible to use both L2 regularization and dropout,
        # but this assignment will only explore one at a time
        if lambd == 0 and keep_prob == 1:
            grads = Utils.backward_propagation(X, Y, cache)
        elif lambd != 0:
            grads = backward_propagation_with_regularization(X, Y, cache, lambd)
        elif keep_prob < 1:
            grads = backward_propagation_with_dropout(X, Y, cache, keep_prob)

        # Update parameters.
        parameters = Utils.update_parameters(parameters, grads, learning_rate)

        # Print the loss every 10000 iterations
        if print_cost and i % 10000 == 0:
            print("Cost after iteration {}: {}".format(i, cost))
        if print_cost and i % 1000 == 0:
            costs.append(cost)

    return parameters, costs

<a name='6'></a>
## 6 - Non-Regularized Model

Let's train the model without any regularization, and observe the accuracy on the train/test sets.

In [ ]:
# Initialize the parameters
iterations = list(range(0, 30000, 1000))

# Load the dataset
data_path = '../data/data.mat'
dataloader = DataLoader(data_path)
train_X, train_Y, test_X, test_Y = dataloader.load_dataset()

# Train the model
parameters, costs = model(train_X, train_Y)

# Plot training costs
Visualization.plot_the_cost(iterations, costs)

# Predict
print("---")
_ = Utils.predict(train_X, train_Y, parameters, True)
_ = Utils.predict(test_X, test_Y, parameters, False)

# Plot decision boundaries
decision_train = Utils.get_decision_boundary(lambda x: Utils.predict_dec(parameters, x.T), train_X, train_Y)
decision_test = Utils.get_decision_boundary(lambda x: Utils.predict_dec(parameters, x.T), test_X, test_Y)

Visualization.plot_decision_boundaries(decision_train, train_X, train_Y, decision_test, test_X, test_Y)

<a name='6-1'></a>
## 6.1 - Analyzing the Non-Regularized Model

The train accuracy is **94.8%** while the test accuracy is **91.5%**.
This is the baseline model (you will observe the impact of regularization on this model).
Analyzing the decision boundary, we can see clearly that the non-regularized model is overfitting the training set. It is fitting the noisy points! Let's now look at two techniques to reduce overfitting.

<a name='7'></a>
## 7 - L2 Regularization \[4 points\]

The standard way to avoid overfitting is called **L2 regularization**.

<a name='7-1'></a>
## 7.1 - Cost Function \[2 points\]

L2 regularization consists of appropriately modifying your cost function, from:
$$J = -\frac{1}{m} \sum\limits_{i = 1}^{m} \large{(}\small  y^{(i)}\log\left(a^{[L](i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right) \large{)}$$
To:
$$J_{regularized} = \small \underbrace{-\frac{1}{m} \sum\limits_{i = 1}^{m} \large{(}\small y^{(i)}\log\left(a^{[L](i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right) \large{)} }_\text{cross-entropy cost} + \underbrace{\frac{1}{m} \frac{\lambda}{2} \sum\limits_l\sum\limits_k\sum\limits_j W_{k,j}^{[l]2} }_\text{L2 regularization cost}$$

Let's modify your cost and observe the consequences.

<b><span style="color:teal">TODO:</span> <b>
<ol>
<li><span style="color:teal">Implement the method <span style="color:#DC143C"><em>compute_cost_with_regularization</em></span> which computes the cost given by the second formula.</span></li>
</ol>

<b><span style="color:#B8860B">Hints:</span> <b>
<dl>
<dd><span style="color:#B8860B">- To calculate $\sum\limits_k\sum\limits_j W_{k,j}^{[l]2}$  , use "np.sum(np.square(Wl))"</span></dd>
<dd><span style="color:#B8860B">- Note that you have to do this for $W^{[1]}$, $W^{[2]}$ and $W^{[3]}$, then sum the three terms and multiply by $ \frac{1}{m} \frac{\lambda}{2} $.</span></dd>
</dl>

In [ ]:
def compute_cost_with_regularization(A3, Y, parameters, lambd):
    """
    Implement the cost function with L2 regularization. See formula (2) above.

    :param A3: post-activation, output of forward propagation, of shape (output size, number of examples)
    :param Y: "true" labels vector, of shape (output size, number of examples)
    :param parameters: python dictionary containing parameters of the model
    :param lambd: regularization hyperparameter, scalar

    :return:
        - cost: value of the regularized loss function (formula (2))
    """

    # Calculate the cross-entropy part of the cost
    cross_entropy_cost = Utils.compute_cost(A3, Y)

    L2_regularization_cost = 0
    #############################################################################
    #                            START OF YOUR CODE                             #
    # TODO:                                                                     #
    #    1) Calculate the L2 regularization cost.                               #
    #                                                                           #
    # Hints:                                                                    #
    #    - Use np.sum(np.square(Wl)) when appropriate.                          #
    #############################################################################


    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################

    cost = cross_entropy_cost + L2_regularization_cost

    return cost

In [ ]:
# Test your code

!python ./tests/test_regularization.py --test_case TestCostL2Regularization

<a name='7-2'></a>
## 7.2 - Backward Propagation with Regularization \[2 points\]

Since we have changed the cost calculation, we have to change the backward propagation as well! All the gradients have to be computed with respect to this new cost.

<b><span style="color:teal">TODO:</span> <b>
<ol>
<li><span style="color:teal">Implement the method <span style="color:#DC143C"><em>backward_propagation_with_regularization</em></span> which applies the changes needed in backward propagation to take regularization into account.</span></li>
</ol>

<b><span style="color:#B8860B">Hints:</span> <b>
<dl>
<dd><span style="color:#B8860B">- The changes only concern dW1, dW2 and dW3.</span></dd>
<dd><span style="color:#B8860B">- For each, you have to add the regularization term's gradient ($\frac{d}{dW} ( \frac{1}{2}\frac{\lambda}{m}  W^2) = \frac{\lambda}{m} W$).</span></dd>
</dl>

In [ ]:
def backward_propagation_with_regularization(X, Y, cache, lambd):
    """
    Implements the backward propagation of our baseline model to which we added an L2 regularization.

    :param X: input dataset, of shape (input size, number of examples).
    :param Y: "true" labels vector, of shape (output size, number of examples).
    :param cache: cache output from forward_propagation().
    :param lambd: regularization hyperparameter, scalar.

    :return:
        - gradients: A dictionary with the gradients with respect to each parameter, activation and pre-activation variables.
    """

    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache

    dZ3 = A3 - Y

    dW3 = 0
    #############################################################################
    #                            START OF YOUR CODE                             #
    # TODO:                                                                     #
    #    1) Include the regularization in the calculation of dW3.               #
    #############################################################################


    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################

    db3 = 1. / m * np.sum(dZ3, axis=1, keepdims=True)

    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))

    dW2 = 0
    #############################################################################
    #                            START OF YOUR CODE                             #
    # TODO:                                                                     #
    #    1) Include the regularization in the calculation of dW2.               #
    #############################################################################


    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################

    db2 = 1. / m * np.sum(dZ2, axis=1, keepdims=True)

    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))

    dW1 = 0
    #############################################################################
    #                            START OF YOUR CODE                             #
    # TODO:                                                                     #
    #    1) Include the regularization in the calculation of dW1.               #
    #############################################################################


    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################

    db1 = 1. / m * np.sum(dZ1, axis=1, keepdims=True)

    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,"dA2": dA2,
                 "dZ2": dZ2, "dW2": dW2, "db2": db2, "dA1": dA1,
                 "dZ1": dZ1, "dW1": dW1, "db1": db1}

    return gradients

In [ ]:
# Test your code

!python ./tests/test_regularization.py --test_case TestBackpropagationWithL2Regularization

<a name='7-3'></a>
## 7.3 - Let's now run the model with L2 regularization

We will use $\lambda = 0.91$ for this.

In [ ]:
# Initialize the parameters
lambd = 0.91
iterations = list(range(0, 30000, 1000))

# Load the dataset
data_path = '../data/data.mat'
dataloader = DataLoader(data_path)
train_X, train_Y, test_X, test_Y = dataloader.load_dataset()

# Train the model
parameters, costs = model(train_X, train_Y, lambd)

# Plot training costs
Visualization.plot_the_cost(iterations, costs)

# Predict
print("---")
_ = Utils.predict(train_X, train_Y, parameters, True)
_ = Utils.predict(test_X, test_Y, parameters, False)

# Plot decision boundaries
decision_train = Utils.get_decision_boundary(lambda x: Utils.predict_dec(parameters, x.T), train_X, train_Y)
decision_test = Utils.get_decision_boundary(lambda x: Utils.predict_dec(parameters, x.T), test_X, test_Y)

Visualization.plot_decision_boundaries(decision_train, train_X, train_Y, decision_test, test_X, test_Y)

<a name='7-4'></a>
## 7.4 - Analyzing L2 regularization
Congrats, the test set accuracy increased to 93%. You have saved the French football team!
The training and test accuracy are both high and very similar. Therefore, you are not overfitting the training data anymore. This can also be seen when analyzing the decision boundary!

**Observations**:
- The value of $\lambda$ is a hyperparameter that you can tune using a dev set.
- L2 regularization makes your decision boundary smoother. If $\lambda$ is too large, it is also possible to "oversmooth", resulting in a model with high bias.


**What is L2-regularization actually doing?**:
- L2-regularization relies on the assumption that a model with small weights is simpler than a model with large weights.
- By penalizing the square values of the weights in the cost function you drive all the weights to smaller values.
- Having large weights becomes too costly for the cost!
- This leads to a smoother model in which the output changes more slowly as the input changes.


**What you should remember:** the implications of L2-regularization on:
- The cost computation: A regularization term is added to the cost.
- The backpropagation function: There are extra terms in the gradients with respect to weight matrices.
- Weights end up smaller ("weight decay"): Weights are pushed to smaller values.

<a name='8'></a>
## 8 - Dropout \[4 points\]

Finally, **dropout** is a widely used regularization technique that is specific to deep learning.

To understand drop-out, consider this conversation with a friend:
- Friend: "Why do you need all these neurons to train your network and classify images?".
- You: "Because each neuron contains a weight and can learn specific features/details/shape of an image. The more neurons I have, the more featurse my model learns!"
- Friend: "I see, but are you sure that your neurons are learning different features and not all the same features?"
- You: "Good point... Neurons in the same layer actually don't talk to each other. It should be definitly possible that they learn the same image features/shapes/forms/details... which would be redundant. There should be a solution."

<div>
<video src="../img/dropout1_kiank.mp4" width="700"/>
</div>

**Dropout randomly shuts down some neurons in each iteration.** Watch these two videos **dropout1_kiank.mp4** and  **dropout2_kiank.mp4** within the "img"-folder to see what this means exactly.
At each iteration, you shut down (= set to zero) each neuron of a layer with probability $1 - keep\_prob$ or keep it with probability $keep\_prob$ (50% here). The dropped neurons don't contribute to the training in both the forward and backward propagations of the iteration.

When you shut some neurons down, you actually modify your model. The idea behind drop-out is that at each iteration, you train a different model that uses only a subset of your neurons. With dropout, your neurons thus become less sensitive to the activation of one other specific neuron, because that other neuron might be shut down at any time.

<a name='8-1'></a>
## 8.1 - Forward Propagation with Dropout \[2 points\]

Implement the forward propagation with dropout. You are using a 3 layer neural network, and will add dropout to the first and second hidden layers. We will not apply dropout to the input layer or output layer.


<b><span style="color:teal">TODO:</span> <b>
<ol>
<li><span style="color:teal">Implement the method <span style="color:#DC143C"><em>forward_propagation_with_dropout</em></span> which includes dropout into the forward pass of the model.</span></li>
<ol>
<li><span style="color:teal">We know that we can create a variable $d^{[1]}$ with the same shape as $a^{[1]}$ using `np.random.rand()` to randomly get numbers between 0 and 1. Here, you will use a vectorized implementation, so create a random matrix $D^{[1]} = [d^{[1](1)} d^{[1](2)} ... d^{[1](m)}] $ of the same dimension as $A^{[1]}$.</span></li>

<li><span style="color:teal">Set each entry of $D^{[1]}$ to be 1 with probability (`keep_prob`), and 0 otherwise.</span></li>

<li><span style="color:teal">Set $A^{[1]}$ to $A^{[1]} * D^{[1]}$. (You are shutting down some neurons). You can think of $D^{[1]}$ as a mask, so that when it is multiplied with another matrix, it shuts down some of the values.</span></li>

<li><span style="color:teal">Divide $A^{[1]}$ by `keep_prob`. By doing this you are assuring that the result of the cost will still have the same expected value as without drop-out. (This technique is also called inverted dropout.)</span></li>
</ol>
</ol>

**Hint:**
Let's say that keep_prob = 0.8, which means that we want to keep about 80% of the neurons and drop out about 20% of them.  We want to generate a vector that has 1's and 0's, where about 80% of them are 1 and about 20% are 0. This python statement: `X = (X < keep_prob).astype(int)` is conceptually the same as this if-else statement (for the simple case of a one-dimensional array) :
```
for i,v in enumerate(x):
    if v < keep_prob:
        x[i] = 1
    else: # v >= keep_prob
        x[i] = 0
```
Note that the `X = (X < keep_prob).astype(int)` works with multi-dimensional arrays, and the resulting output preserves the dimensions of the input array.

Also note that without using `.astype(int)`, the result is an array of booleans `True` and `False`, which Python automatically converts to 1 and 0 if we multiply it with numbers.  (However, it's better practice to convert data into the data type that we intend, so try using `.astype(int)`.)

In [ ]:
def forward_propagation_with_dropout(X, parameters, keep_prob = 0.5):
    """
    Implements the forward propagation: LINEAR -> RELU + DROPOUT -> LINEAR -> RELU + DROPOUT -> LINEAR -> SIGMOID.

    :param X: input dataset, of shape (2, number of examples).
    :param parameters: python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3".
    :param keep_prob: probability of keeping a neuron active during drop-out, scalar.

    :return:
        - A3: last activation value, output of the forward propagation, of shape (1,1).
        - cache: tuple, information stored for computing the backward propagation.
    """

    np.random.seed(1)

    # retrieve parameters
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]

    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = np.dot(W1, X) + b1
    A1 = Utils.relu(Z1)

    D1 = None
    #############################################################################
    #                            START OF YOUR CODE                             #
    # TODO:                                                                     #
    #    1) Initialize matrix D1                                                #
    #    2) Convert entries of D1 to 0 or 1 (using keep_prob as the threshold)  #
    #    3) Shut down some neurons of A1.                                       #
    #    4) Scale the value of neurons that haven't been shut down.             #
    #############################################################################


    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################

    Z2 = np.dot(W2, A1) + b2
    A2 = Utils.relu(Z2)

    D2 = None
    #############################################################################
    #                            START OF YOUR CODE                             #
    # TODO:                                                                     #
    #    1) Initialize matrix D2                                                #
    #    2) Convert entries of D2 to 0 or 1 (using keep_prob as the threshold)  #
    #    3) Shut down some neurons of A2.                                       #
    #    4) Scale the value of neurons that haven't been shut down.             #
    #############################################################################


    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################

    Z3 = np.dot(W3, A2) + b3
    A3 = Utils.sigmoid(Z3)

    cache = (Z1, D1, A1, W1, b1, Z2, D2, A2, W2, b2, Z3, A3, W3, b3)

    return A3, cache

In [ ]:
# Test your code

!python ./tests/test_regularization.py --test_case TestForwardPropagationWithDropout

<a name='8-2'></a>
## 8.2 - Backward Propagation with Dropout \[2 points\]

Implement the backward propagation with dropout. As before, you are training a 3 layer network. Add dropout to the first and second hidden layers, using the masks $D^{[1]}$ and $D^{[2]}$ stored in the cache.


<b><span style="color:teal">TODO:</span> <b>
<ol>
<li><span style="color:teal">Implement the method <span style="color:#DC143C"><em>backward_propagation_with_dropout</em></span> which includes dropout into the backward pass of the model.</span></li>
<ol>
<li><span style="color:teal">You had previously shut down some neurons during forward propagation, by applying a mask $D^{[1]}$ to `A1`. In backpropagation, you will have to shut down the same neurons, by reapplying the same mask $D^{[1]}$ to `dA1`.</span></li>

<li><span style="color:teal">During forward propagation, you had divided `A1` by `keep_prob`. In backpropagation, you'll therefore have to divide `dA1` by `keep_prob` again (the calculus interpretation is that if $A^{[1]}$ is scaled by `keep_prob`, then its derivative $dA^{[1]}$ is also scaled by the same `keep_prob`).</span></li>
</ol>
</ol>

In [ ]:
def backward_propagation_with_dropout(X, Y, cache, keep_prob):
    """
    Implements the backward propagation of our baseline model to which we added dropout.

    Arguments:
    :param X: input dataset, of shape (2, number of examples).
    :param Y: "true" labels vector, of shape (output size, number of examples).
    :param cache: cache output from forward_propagation_with_dropout().
    :param keep_prob: probability of keeping a neuron active during drop-out, scalar.

    :return:
        - gradients: A dictionary with the gradients with respect to each parameter, activation and pre-activation variables.
    """

    m = X.shape[1]
    (Z1, D1, A1, W1, b1, Z2, D2, A2, W2, b2, Z3, A3, W3, b3) = cache

    dZ3 = A3 - Y
    dW3 = 1./m * np.dot(dZ3, A2.T)
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims=True)
    dA2 = np.dot(W3.T, dZ3)

    #############################################################################
    #                            START OF YOUR CODE                             #
    # TODO:                                                                     #
    #    1) Apply mask D2 to shut down the same neurons of A2 as during the     #
    #       forward propagation.                                                #
    #    2) Scale the value of neurons that haven't been shut down.             #
    #############################################################################


    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################

    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1./m * np.dot(dZ2, A1.T)
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims=True)

    dA1 = np.dot(W2.T, dZ2)

    #############################################################################
    #                            START OF YOUR CODE                             #
    # TODO:                                                                     #
    #    1) Apply mask D1 to shut down the same neurons of A1 as during the     #
    #       forward propagation.                                                #
    #    2) Scale the value of neurons that haven't been shut down.             #
    #############################################################################


    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################

    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 1./m * np.sum(dZ1, axis=1, keepdims=True)

    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,"dA2": dA2,
                 "dZ2": dZ2, "dW2": dW2, "db2": db2, "dA1": dA1,
                 "dZ1": dZ1, "dW1": dW1, "db1": db1}

    return gradients

In [ ]:
# Test your code

!python ./tests/test_regularization.py --test_case TestBackpropagationWithDropout

<a name='8-3'></a>
## 8.3 - Let's now run the model with Dropout

We will use $keep\_prob = 0.86$ for this. It means at every iteration you shut down each neurons of layer 1 and 2 with 14% probability. Our function `model()` will now call:
- `forward_propagation_with_dropout` instead of `forward_propagation`.
- `backward_propagation_with_dropout` instead of `backward_propagation`.

In [ ]:
# Initialize the parameters
keep_prob = 0.86
learning_rate = 0.3
iterations = list(range(0, 30000, 1000))

# Load the dataset
data_path = '../data/data.mat'
dataloader = DataLoader(data_path)
train_X, train_Y, test_X, test_Y = dataloader.load_dataset()

# Train the model
parameters, costs = model(train_X, train_Y, keep_prob=keep_prob, learning_rate=learning_rate)

# Plot training costs
Visualization.plot_the_cost(iterations, costs)

# Predict
print("---")
_ = Utils.predict(train_X, train_Y, parameters, True)
_ = Utils.predict(test_X, test_Y, parameters, False)

# Plot decision boundaries
decision_train = Utils.get_decision_boundary(lambda x: Utils.predict_dec(parameters, x.T), train_X, train_Y)
decision_test = Utils.get_decision_boundary(lambda x: Utils.predict_dec(parameters, x.T), test_X, test_Y)

Visualization.plot_decision_boundaries(decision_train, train_X, train_Y, decision_test, test_X, test_Y)

<a name='8-4'></a>
## 8.4 - Analyzing Dropout
Dropout works great! The test accuracy has increased again (to 95%)! Your model is not overfitting the training set and does a great job on the test set. The French football team will be forever grateful to you!

**Note**:
- A **common mistake** when using dropout is to use it both in training and testing. You should use dropout (randomly eliminate nodes) only in training.
- Deep learning frameworks like [PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html), [tensorflow](https://www.tensorflow.org/api_docs/python/tf/nn/dropout), [PaddlePaddle](http://doc.paddlepaddle.org/release_doc/0.9.0/doc/ui/api/trainer_config_helpers/attrs.html), [keras](https://keras.io/layers/core/#dropout) or [caffe](http://caffe.berkeleyvision.org/tutorial/layers/dropout.html) come with a dropout layer implementation to ease or lives.

**What you should remember about dropout:**
- Dropout is a regularization technique.
- You only use dropout during training. Don't use dropout (randomly eliminate nodes) during test time.
- Apply dropout both during forward and backward propagation.
- During training time, divide each dropout layer by keep_prob to keep the same expected value for the activations. For example, if keep_prob is 0.5, then we will on average shut down half the nodes, so the output will be scaled by 0.5 since only the remaining half are contributing to the solution. Dividing by 0.5 is equivalent to multiplying by 2. Hence, the output now has the same expected value. You can check that this works even when keep_prob is other values than 0.5.

<a name='9'></a>
## 9 - Conclusions

**Here are the results of our three models**:

<table>
    <tr>
        <td>
        <b>model</b>
        </td>
        <td>
        <b>train accuracy</b>
        </td>
        <td>
        <b>test accuracy</b>
        </td>
    </tr>
        <td>
        3-layer NN without regularization
        </td>
        <td>
        95%
        </td>
        <td>
        91.5%
        </td>
    <tr>
        <td>
        3-layer NN with L2-regularization
        </td>
        <td>
        94%
        </td>
        <td>
        93%
        </td>
    </tr>
    <tr>
        <td>
        3-layer NN with dropout
        </td>
        <td>
        93%
        </td>
        <td>
        95%
        </td>
    </tr>
</table>

<a id='10'></a>
## 10 - End of Exercise ##

<div>
<img src="../img/memes/meme_congrats_02.png" width="700"/>
</div>

Created with and licensed under [Adobe Express](https://www.adobe.com/de/express/)